## SummaC

In [ ]:
# Restart runtime before running this!

!pip uninstall -y transformers accelerate huggingface-hub
!pip install numpy==1.26.0
!pip install torch==2.0.0
!pip install transformers==4.30.0
!pip install protobuf==3.20.3
!pip install huggingface_hub==0.17.0
!pip install summac

  Using cached transformers-4.30.0-py3-none-any.whl.metadata (113 kB)
  Using cached huggingface_hub-0.33.0-py3-none-any.whl.metadata (14 kB)
Using cached transformers-4.30.0-py3-none-any.whl (7.2 MB)
Using cached huggingface_hub-0.33.0-py3-none-any.whl (514 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.15.2 requires accelerate>=0.21.0, which is not installed.
summac 0.0.4 requires huggingface-hub<=0.17.0, but you have huggingface-hub 0.33.0 which is incompatible.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.30.0 which is incompatible.
  Using cached huggingface_hub-0.17.0-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-0.17.0-py3-none-any.whl (294 kB)
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.33.0
    Uninstalling huggingface-hub-0.33.0

In [ ]:
import ast
import pandas as pd
# from summac.model_summac import SummaCZS, SummaCConv

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
RESULT_PATH = 'Retrieved_Results_of_Combined_QA_Pairs_by_jinaai-jina-embeddings-v3.xlsx'
results = pd.read_excel(RESULT_PATH)

In [ ]:
RESPONSE_PATH = 'Response_of_Combined_QA_Pairs_by_jinaai-jina-embeddings-v3.xlsx'
response = pd.read_excel(RESPONSE_PATH)

In [ ]:
results["Results"] = results["Results"].apply(ast.literal_eval)

In [ ]:
results["Text"] = results["Results"].apply(lambda x: "\n\n".join(list(zip(*x))[1]))

In [ ]:
results["Text"][0]

'Question 8. When Should Neuromodulation Treatments be Used? > Q.8.b. What Noninvasive Neuromodulation Treatments are Available? > Electroconvulsive Therapy > paragraph id: 216\n\nAn acute course of ECT (Electroconvulsive therapy) usually requires 6 to 12 sessions, with no difference in response outcomes between sessions given twice or thrice weekly (Level 2) . Refinements in the delivery of ECT (Electroconvulsive therapy) have focused on electrode placement and stimulus energy, based on the waveform of electrical stimulus and seizure threshold (the minimum amount of current required to produce a measurable seizure) ( Table 8.1 ). Ultrabrief pulse (∼0.3 ms) waveforms with right unilateral electrode placement are slightly less effective than brief pulse (∼1.0 ms) but appear to have fewer cognitive adverse effects (see below). In contrast, bitemporal brief pulse ECT (Electroconvulsive therapy) may have greater efficacy than other protocols but carries a higher cognitive burden. Furthermo

In [ ]:
response["Response"][0]

'### Electroconvulsive Therapy (ECT) Overview\n* **Definition:** Electroconvulsive therapy (**ECT**) involves the delivery of an electrical stimulus via electrodes placed on the scalp, resulting in the induction of a brief generalized seizure.\n* **Administration:** **ECT** is delivered under general anesthesia and after a muscle relaxant has been administered to minimize the physical manifestations of the seizure and its potential complications.\n* **Indications:** **ECT** remains one of the most effective treatment options for patients with treatment-resistant depression (**TRD**).\n\n### When to Use ECT\n* **First Line of Treatment:** \n  + **ECT protocol:** Brief Pulse, bifrontal (at 1.5 times seizure threshold) - **Level of evidence:** Level 1\n  + **ECT protocol:** Brief Pulse, right unilateral (at 6 times seizure threshold) - **Level of evidence:** Level 1\n* **Second Line of Treatment:**\n  + **ECT protocol:** Brief Pulse, bitemporal (at 1.5 times seizure threshold) - **Level o

In [ ]:
model_conv = SummaCConv(
    models=["vitc"],
    bins='percentile',
    granularity="sentence",
    nli_labels="e",
    device="cuda",
    # start_file="./summac_conv_vitc_sent_perc_e.bin",  # your downloaded path
    agg="mean"
)

In [ ]:
docs = results["Text"].to_list()

In [ ]:
docs

['Question 8. When Should Neuromodulation Treatments be Used? > Q.8.b. What Noninvasive Neuromodulation Treatments are Available? > Electroconvulsive Therapy > paragraph id: 216\n\nAn acute course of ECT (Electroconvulsive therapy) usually requires 6 to 12 sessions, with no difference in response outcomes between sessions given twice or thrice weekly (Level 2) . Refinements in the delivery of ECT (Electroconvulsive therapy) have focused on electrode placement and stimulus energy, based on the waveform of electrical stimulus and seizure threshold (the minimum amount of current required to produce a measurable seizure) ( Table 8.1 ). Ultrabrief pulse (∼0.3 ms) waveforms with right unilateral electrode placement are slightly less effective than brief pulse (∼1.0 ms) but appear to have fewer cognitive adverse effects (see below). In contrast, bitemporal brief pulse ECT (Electroconvulsive therapy) may have greater efficacy than other protocols but carries a higher cognitive burden. Furtherm

In [ ]:
summaries = response["Response"].to_list()
summaries

['### Electroconvulsive Therapy (ECT) Overview\n* **Definition:** Electroconvulsive therapy (**ECT**) involves the delivery of an electrical stimulus via electrodes placed on the scalp, resulting in the induction of a brief generalized seizure.\n* **Administration:** **ECT** is delivered under general anesthesia and after a muscle relaxant has been administered to minimize the physical manifestations of the seizure and its potential complications.\n* **Indications:** **ECT** remains one of the most effective treatment options for patients with treatment-resistant depression (**TRD**).\n\n### When to Use ECT\n* **First Line of Treatment:** \n  + **ECT protocol:** Brief Pulse, bifrontal (at 1.5 times seizure threshold) - **Level of evidence:** Level 1\n  + **ECT protocol:** Brief Pulse, right unilateral (at 6 times seizure threshold) - **Level of evidence:** Level 1\n* **Second Line of Treatment:**\n  + **ECT protocol:** Brief Pulse, bitemporal (at 1.5 times seizure threshold) - **Level 

In [ ]:
scores_list = []
for doc, summ in zip(docs, summaries):
    score = model_conv.score([doc], [summ])  # use batch of size 1
    scores_list.append(score)

/usr/local/lib/python3.11/dist-packages/summac/model_summac.py:298: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  histograms = torch.FloatTensor(histograms).to(self.device)


In [ ]:
document = """Scientists are studying Mars to learn about the Red Planet and find landing sites for future missions.
One possible site, known as Arcadia Planitia, is covered instrange sinuous features.
The shapes could be signs that the area is actually made of glaciers, which are large masses of slow-moving ice.
Arcadia Planitia is in Mars' northern lowlands."""

summary1 = "There are strange shape patterns on Arcadia Planitia. The shapes could indicate the area might be made of glaciers. This makes Arcadia Planitia ideal for future missions."

print("[Summary 1] SummacConv score: %.3f" % (score_conv1["scores"][0])) # [Summary 1] SummaCZS Score: 0.582; SummacConv score: 0.536

[Summary 1] SummacConv score: 0.595


/usr/local/lib/python3.11/dist-packages/summac/model_summac.py:298: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  histograms = torch.FloatTensor(histograms).to(self.device)


In [ ]:
scores = [s["scores"][0] for s in scores_list]
scores

[0.9898231625556946,
 0.9980212450027466,
 0.9962021708488464,
 0.9963675737380981,
 0.9974834322929382,
 0.9764530062675476,
 0.9903106093406677,
 0.9340049028396606,
 0.9924701452255249,
 0.9747353196144104,
 0.8911606073379517,
 0.9914051294326782,
 0.9865062832832336,
 0.9858924746513367,
 0.9730114340782166,
 0.9956037998199463,
 0.9700866341590881,
 0.9934142827987671,
 0.994976818561554,
 0.9913275837898254,
 0.968894362449646,
 0.9998642206192017,
 0.9789028167724609,
 0.9972220659255981,
 0.9963833093643188,
 0.9920400381088257,
 0.9944711923599243,
 0.9964016675949097,
 0.9872487187385559,
 0.9882444143295288,
 0.9814321994781494,
 0.9868626594543457,
 0.9924006462097168,
 0.9964635968208313,
 0.9940962791442871,
 0.9947444200515747,
 0.9566365480422974,
 0.9980018734931946,
 0.9081884622573853,
 0.989083468914032,
 0.9261078238487244,
 0.9528997540473938,
 0.9901782274246216,
 0.9744873046875,
 0.9908851981163025,
 0.9997950196266174,
 0.9939464926719666,
 0.9961994290351868

In [ ]:
score_df = pd.DataFrame.from_dict({"Query": response["Question"].to_list(), "Score": scores})

In [ ]:
score_df.to_csv("summac_scores.csv", index=False)

In [ ]:
score_df.describe()

,Score
count,134.000000
mean,0.981180
std,0.037629
min,0.681312
25%,0.978988
50%,0.992265
75%,0.996417
max,0.999974


## QuestEval

In [ ]:
# Install system dependencies
!apt-get install -y git-lfs

# Clone the working fork of QuestEval (official repo often breaks)
!git clone https://github.com/ThomasScialom/QuestEval.git
%cd QuestEval

# Use updated pip and install requirements manually
!pip install --upgrade pip setuptools wheel

# Uninstall pyarrow to avoid conflicts
!pip uninstall -y pyarrow

# Download the requirements.txt file, remove the spacy and pyarrow version constraints, and save it
!wget -O requirements.txt https://raw.githubusercontent.com/ThomasScialom/QuestEval/main/requirements.txt
!sed -i '/^spacy==/d' requirements.txt
!sed -i '/^pyarrow==/d' requirements.txt
!sed -i '/^pyarrow<4.0.0/d' requirements.txt


# Install requirements using the modified file, without using cache
!pip install -r requirements.txt --no-cache-dir

# Install QuestEval itself
!pip install .

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Cloning into 'QuestEval'...
remote: Enumerating objects: 634, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 634 (delta 69), reused 65 (delta 65), pack-reused 543 (from 1)
Receiving objects: 100% (634/634), 897.70 KiB | 1.95 MiB/s, done.
Resolving deltas: 100% (275/275), done.
/content/QuestEval/QuestEval/QuestEval/QuestEval
Found existing installation: pyarrow 18.1.0
Uninstalling pyarrow-18.1.0:
  Successfully uninstalled pyarrow-18.1.0
--2025-06-13 11:44:00--  https://raw.githubusercontent.com/ThomasScialom/QuestEval/main/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw

In [ ]:
!pip install -q transformers

In [ ]:
from transformers import pipeline

# Load NLI model
nli = pipeline("text-classification", model="roberta-large-mnli", framework="pt", device=0)

# Define your question and answer
question = "Where is the Eiffel Tower located?"
answer = "The Eiffel Tower is in Paris."

# Format the input for NLI
input_text = f"{question} </s> {answer}"

# Run the NLI model
result = nli(input_text)
print("Prediction:", result[0]['label'])
print("Confidence:", result[0]['score'])

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Prediction: NEUTRAL
Confidence: 0.9574085474014282


In [ ]:
queries = response["Question"].to_list()

In [ ]:
pred = []
for q, s in zip(queries, summaries):
    input_text = f"{q} </s> {s}"
    result = nli(input_text, truncation=True)
    pred.append({"Prediction": result[0]['label'], "Confidence": result[0]['score']})

/usr/local/lib/python3.11/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
pred

[{'Prediction': 'CONTRADICTION', 'Confidence': 0.8170391321182251},
 {'Prediction': 'NEUTRAL', 'Confidence': 0.9410357475280762},
 {'Prediction': 'CONTRADICTION', 'Confidence': 0.8048454523086548},
 {'Prediction': 'NEUTRAL', 'Confidence': 0.9886642098426819},
 {'Prediction': 'NEUTRAL', 'Confidence': 0.994987964630127},
 {'Prediction': 'ENTAILMENT', 'Confidence': 0.45728257298469543},
 {'Prediction': 'NEUTRAL', 'Confidence': 0.6159121990203857},
 {'Prediction': 'NEUTRAL', 'Confidence': 0.9953591227531433},
 {'Prediction': 'CONTRADICTION', 'Confidence': 0.5606798529624939},
 {'Prediction': 'NEUTRAL', 'Confidence': 0.9843820929527283},
 {'Prediction': 'NEUTRAL', 'Confidence': 0.979569137096405},
 {'Prediction': 'NEUTRAL', 'Confidence': 0.5427039265632629},
 {'Prediction': 'NEUTRAL', 'Confidence': 0.9798889756202698},
 {'Prediction': 'CONTRADICTION', 'Confidence': 0.8001131415367126},
 {'Prediction': 'NEUTRAL', 'Confidence': 0.9893869161605835},
 {'Prediction': 'NEUTRAL', 'Confidence': 0.9